In [35]:
import sage_data_client
import numpy as np
from pathlib import Path
import re
import requests
import numpy as np
import cv2

In [36]:
def download_image(vsn, url):
    image_req = requests.get(url)
    status_code = image_req.status_code
    if status_code == 200:
        image = np.asarray(bytearray(image_req.content))
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    else:
        print('Invalid URL: ' + url)
        print('Status code: ' + str(status_code))
        return

    basename = url.split("/")[-1]
    filename = f"attention/{vsn}/{basename}"
    Path(filename).parent.mkdir(parents=True, exist_ok=True)
    cv2.imwrite(filename, image)

def get_images(df_all,image_type="sample_current.jpg"):
    uploads = df_all[df_all.name == "upload"]
    if len(uploads) == 0:
        print("No data")
        return
    for (vsn, ts), df in df_all.groupby(["meta.vsn", "timestamp"]):
        try:
            url = df[df["meta.filename"] == image_type].value.iloc[0]
        except IndexError:
            print("skip", url)
            continue
        url = re.sub("/smoke-detector-\d+/", "/sage/", url)
        download_image(vsn, url)

## Get data from Sage with data client
- UI browser for another way to explore the data: https://portal.sagecontinuum.org/data-browser?window=d&nodes=W01D&apps=wildfire-smoke-detection%3A0.5.0&start=2022-04-15T07%3A25%3A00.000Z

In [39]:
df = sage_data_client.query(
        start="2022-04-15 10:00:00 CST",
        end="2022-04-16 22:00:00 CST",
        filter={
            "plugin": "wildfire-smoke-detection:0.5.0",
            "vsn": 'W01F'
        }
)

/usr/local/lib/python3.9/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname CST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


### The tiles are provided through `env.smoke.tile_probs`

In [40]:
df[df.name == 'env.smoke.tile_probs']

,timestamp,name,value,meta.camera,meta.host,meta.job,meta.node,meta.plugin,meta.task,meta.vsn,meta.filename
0,2022-04-15 18:37:57.378637152+00:00,env.smoke.tile_probs,"[[[0.0012372826458886266], [0.0011814879253506...",bottom,000048b02d15bc64.ws-nxcore,smoke-detector-1650046302,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,NaN
1,2022-04-15 18:43:35.130604940+00:00,env.smoke.tile_probs,"[[[0.0008699279278516769], [0.0008772349683567...",bottom,000048b02d15bc64.ws-nxcore,smoke-detector-1650047873,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,NaN
2,2022-04-15 18:47:15.184480372+00:00,env.smoke.tile_probs,"[[[0.001043850788846612], [0.00097571109654381...",bottom,000048b02d15bc64.ws-nxcore,smoke-detector-1650048210,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,NaN
3,2022-04-15 18:47:59.687699436+00:00,env.smoke.tile_probs,"[[[0.0011268648086115718], [0.0254153646528720...",bottom,000048b02d15bc64.ws-nxcore,smoke-detector-1650048269,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,NaN
4,2022-04-15 18:49:56.373683430+00:00,env.smoke.tile_probs,"[[[0.0010996367782354355], [0.0009101392934098...",bottom,000048b02d15bc64.ws-nxcore,smoke-detector-1650048430,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1015,2022-04-15 20:05:38.106231013+00:00,env.smoke.tile_probs,"[[[0.0007616269867867231], [0.0007666455931030...",bottom,000048b02d15bc64.ws-nxcore,NaN,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,NaN
1016,2022-04-15 20:27:13.708357617+00:00,env.smoke.tile_probs,"[[[0.0007323962054215372], [0.0007180256652645...",bottom,000048b02d15bc64.ws-nxcore,NaN,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,NaN
1017,2022-04-15 21:02:52.099359372+00:00,env.smoke.tile_probs,"[[[0.0007217078818939626], [0.0007225667359307...",bottom,000048b02d15bc64.ws-nxcore,NaN,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,NaN
1018,2022-04-15 21:46:18.139625795+00:00,env.smoke.tile_probs,"[[[0.1044333428144455], [0.24027134478092194],...",bottom,000048b02d15bc64.ws-nxcore,NaN,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,NaN


### The images used for the inference are in the `name` column with value `upload`
- `meta.filename` has two values: `sample_current.jpg` and `sample_previous.jpg` which are the images used for the inference 

In [41]:
df[df.name == 'upload']

,timestamp,name,value,meta.camera,meta.host,meta.job,meta.node,meta.plugin,meta.task,meta.vsn,meta.filename
1020,2022-04-15 18:37:57.378637152+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,NaN,000048b02d15bc64.ws-nxcore,smoke-detector-1650046302,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,sample_current.jpg
1021,2022-04-15 18:43:35.130604940+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,NaN,000048b02d15bc64.ws-nxcore,smoke-detector-1650047873,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,sample_current.jpg
1022,2022-04-15 18:47:15.184480372+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,NaN,000048b02d15bc64.ws-nxcore,smoke-detector-1650048210,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,sample_current.jpg
1023,2022-04-15 18:47:59.687699436+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,NaN,000048b02d15bc64.ws-nxcore,smoke-detector-1650048269,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,sample_current.jpg
1024,2022-04-15 18:49:56.373683430+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,NaN,000048b02d15bc64.ws-nxcore,smoke-detector-1650048430,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,sample_current.jpg
...,...,...,...,...,...,...,...,...,...,...,...
3055,2022-04-15 20:05:38.106231013+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,NaN,000048b02d15bc64.ws-nxcore,NaN,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,sample_previous.jpg
3056,2022-04-15 20:27:13.708357617+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,NaN,000048b02d15bc64.ws-nxcore,NaN,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,sample_previous.jpg
3057,2022-04-15 21:02:52.099359372+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,NaN,000048b02d15bc64.ws-nxcore,NaN,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,sample_previous.jpg
3058,2022-04-15 21:46:18.139625795+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,NaN,000048b02d15bc64.ws-nxcore,NaN,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,sample_previous.jpg


In [42]:
df[(df.name == 'upload') & (df['meta.filename'] == 'sample_current.jpg')]

,timestamp,name,value,meta.camera,meta.host,meta.job,meta.node,meta.plugin,meta.task,meta.vsn,meta.filename
1020,2022-04-15 18:37:57.378637152+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,NaN,000048b02d15bc64.ws-nxcore,smoke-detector-1650046302,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,sample_current.jpg
1021,2022-04-15 18:43:35.130604940+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,NaN,000048b02d15bc64.ws-nxcore,smoke-detector-1650047873,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,sample_current.jpg
1022,2022-04-15 18:47:15.184480372+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,NaN,000048b02d15bc64.ws-nxcore,smoke-detector-1650048210,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,sample_current.jpg
1023,2022-04-15 18:47:59.687699436+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,NaN,000048b02d15bc64.ws-nxcore,smoke-detector-1650048269,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,sample_current.jpg
1024,2022-04-15 18:49:56.373683430+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,NaN,000048b02d15bc64.ws-nxcore,smoke-detector-1650048430,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,sample_current.jpg
...,...,...,...,...,...,...,...,...,...,...,...
2035,2022-04-15 20:05:38.106231013+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,NaN,000048b02d15bc64.ws-nxcore,NaN,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,sample_current.jpg
2036,2022-04-15 20:27:13.708357617+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,NaN,000048b02d15bc64.ws-nxcore,NaN,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,sample_current.jpg
2037,2022-04-15 21:02:52.099359372+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,NaN,000048b02d15bc64.ws-nxcore,NaN,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,sample_current.jpg
2038,2022-04-15 21:46:18.139625795+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,NaN,000048b02d15bc64.ws-nxcore,NaN,000048b02d15bc64,wildfire-smoke-detection:0.5.0,smoke-detector,W01F,sample_current.jpg


### Three nodes were used for the controlled fire, so the grouping should be done by node (vsn and timestamp)
- Note that node `W01F` was in the direct sight of the fires. Still need to verify if the other nodes (`W038` and `W01D`) are ever in the direct sight of the fires at any point of the controlled fire.

In [43]:
for (vsn, ts), _ in df.groupby(["meta.vsn", "timestamp"]):
    print('vsn: ' + str(vsn))
    print('timestamp: ' + str(ts))

vsn: W01F
timestamp: 2022-04-15 18:20:12.568317404+00:00
vsn: W01F
timestamp: 2022-04-15 18:37:57.378637152+00:00
vsn: W01F
timestamp: 2022-04-15 18:43:35.130604940+00:00
vsn: W01F
timestamp: 2022-04-15 18:44:34.317260851+00:00
vsn: W01F
timestamp: 2022-04-15 18:47:15.184480372+00:00
vsn: W01F
timestamp: 2022-04-15 18:47:59.687699436+00:00
vsn: W01F
timestamp: 2022-04-15 18:49:56.373683430+00:00
vsn: W01F
timestamp: 2022-04-15 18:50:38.880846073+00:00
vsn: W01F
timestamp: 2022-04-15 18:52:16.320542930+00:00
vsn: W01F
timestamp: 2022-04-15 18:53:17.038346235+00:00
vsn: W01F
timestamp: 2022-04-15 18:55:05.620313445+00:00
vsn: W01F
timestamp: 2022-04-15 18:56:47.880735850+00:00
vsn: W01F
timestamp: 2022-04-15 18:58:03.448053651+00:00
vsn: W01F
timestamp: 2022-04-15 18:59:39.222571814+00:00
vsn: W01F
timestamp: 2022-04-15 19:01:17.900443690+00:00
vsn: W01F
timestamp: 2022-04-15 19:02:56.762370173+00:00
vsn: W01F
timestamp: 2022-04-15 19:04:57.542735523+00:00
vsn: W01F
timestamp: 2022-04-15

### Download all the images from the query

In [44]:
print('Length of df: ' + str(len(df)))
sage_nodes = list(df['meta.vsn'].unique())
print('Sage nodes:' + str(sage_nodes))

Length of df: 3060
Sage nodes:['W01F']


In [45]:
get_images(df)

Invalid URL: https://storage.sagecontinuum.org/api/v1/data//sage-smoke-detector-0.5.0/000048b02d15bc64/1650046812568317404-sample_current.jpg
Status code: 400
Invalid URL: https://storage.sagecontinuum.org/api/v1/data//sage-smoke-detector-0.5.0/000048b02d15bc64/1650048274317260851-sample_current.jpg
Status code: 400
Invalid URL: https://storage.sagecontinuum.org/api/v1/data//sage-smoke-detector-0.5.0/000048b02d15bc64/1650048797038346235-sample_current.jpg
Status code: 400
Invalid URL: https://storage.sagecontinuum.org/api/v1/data//sage-smoke-detector-0.5.0/000048b02d15bc64/1650049376762370173-sample_current.jpg
Status code: 400
Invalid URL: https://storage.sagecontinuum.org/api/v1/data//sage-smoke-detector-0.5.0/000048b02d15bc64/1650053138106231013-sample_current.jpg
Status code: 400
Invalid URL: https://storage.sagecontinuum.org/api/v1/data//sage-smoke-detector-0.5.0/000048b02d15bc64/1650054433708357617-sample_current.jpg
Status code: 400
Invalid URL: https://storage.sagecontinuum.org